In [ ]:
import os
import pandas as pd
import numpy as np
import sys
import os

def find_and_append_module_path():
    current_dir = os.getcwd()
    substring_to_find = 'statsfaction'
    index = current_dir.rfind(substring_to_find)
    
    if index != -1:
        # Extract the directory path up to and including the last "mypath" occurrence
        new_dir = current_dir[:index + (len(substring_to_find))]

        # Change the current working directory to the new directory
        os.chdir(new_dir)
        sys.path.append(new_dir)
        # Verify the new current directory
        print("New current directory:", os.getcwd())
    else:
        print("No 'mypath' found in the current directory")

find_and_append_module_path()
os.getcwd()


In [ ]:
from prediction_prime_overall.src.predict import predict_and_explain_players, analyze_individual_ID
from prediction_prime_overall.src.eval import create_polar_plot
import prediction_prime_overall.config as CONFIG

# Import your group to analyze

In [ ]:
sql_prospect = f"""
SELECT min(Age) as potential_age,* FROM  fifa WHERE  FIFA = 2024 and Age < 23 and potential > 70 and overall <90
GROUP BY ID
order by potential DESC;
"""


import pandas as pd
import sqlite3
DATABASE_PATH = "data/sport_analytics/database/football.db"
# Step 1: Establish a database connection
conn = sqlite3.connect(CONFIG.DATABASE)

df_prospect = pd.read_sql_query(sql_prospect, conn)

conn.close()

# Seiten wurden umgestellt und Feld hat jetzt neuen Inhalt
# df_prospect['Position'] = [value.strip() for item in df_prospect["Position"] for value in item.split(',')]
# df_prospect['Position'] = [i.replace(' ',',') for i in df_prospect['Position']]

df_prospect.head()

## Choose a from a Set of trained models

In [ ]:
import pickle
NAME = "offense_potential_forward_min_pot_min_prime"
ACTUAL_MODEL = "offense_potential_forward_general_age"
with open(f"{CONFIG.MODELS}/{NAME}.pkl", "rb") as file:
    loaded_object = pickle.load(file)

chosen_object = loaded_object[ 'Gradient Boosting Regressor']
model = chosen_object['Model']
scaler = chosen_object['Scaler']
attributes = chosen_object['attributes']
explainer = chosen_object['explainer']
print(chosen_object['Classification Report'])
# print(chosen_object['AUC'])

In [ ]:
# Mathys Tel
player_attr = {	
    "Age":18,
    "Crossing":56,
    "Finishing":78,
    "Heading accuracy":61,
    "ShortPassing":68,
     "Volleys":63,
    
    "Dribbling":79,
    "FKAccuracy":56,
    "LongPassing":59,
    "BallControl":77,


    "Acceleration":79,
    "Reactions":60,

     'Aggression':48,
     "Positioning":73,

     "Vision":70,
    'Composure':74,

    "ShotPower":82,

    "Jumping":75,
    "Strength":67,
    "LongShots":73,
}

# Outer Midfielder

In [ ]:
loaded_object = pd.read_pickle("prediction_prime_overall/data/models/outside_midfielder_prime.pkl")[ 'Gradient Boosting Regressor']
model = loaded_object['Model']
scaler = loaded_object['Scaler']
print(model.feature_names_in_)



input_model = pd.DataFrame(pd.Series(player_attr)).T
input_model = scaler.transform(input_model[model.feature_names_in_])
print(model.predict(input_model))
explainer = loaded_object['explainer']
pd.DataFrame(index=model.feature_names_in_, data = explainer.shap_values(input_model)[0])

## Winger

In [ ]:
loaded_winger = pd.read_pickle("prediction_prime_overall/data/models/winger_prime.pkl")[ 'Gradient Boosting Regressor']
model = loaded_winger['Model']
scaler = loaded_winger['Scaler']
print(model.feature_names_in_)



input_model = pd.DataFrame(pd.Series(player_attr)).T
input_model = scaler.transform(input_model[model.feature_names_in_])
print(model.predict(input_model))
explainer = loaded_winger['explainer']
pd.DataFrame(index=model.feature_names_in_, data = explainer.shap_values(input_model)[0])

## Center Forward

In [ ]:
loaded_object = pd.read_pickle("prediction_prime_overall/data/models/forward_prime.pkl")[ 'Gradient Boosting Regressor']
model = loaded_object['Model']
scaler = loaded_object['Scaler']
print(model.feature_names_in_)


input_model = pd.DataFrame(pd.Series(player_attr)).T
input_model = scaler.transform(input_model[model.feature_names_in_])
print(model.predict(input_model))

explainer = loaded_object['explainer']
pd.DataFrame(index=model.feature_names_in_, data = explainer.shap_values(input_model)[0])

## Attacking Midfielder

In [ ]:
loaded_winger = pd.read_pickle("prediction_prime_overall/data/models/attacking_midfielder_prime.pkl")[ 'Gradient Boosting Regressor']
model = loaded_winger['Model']
scaler = loaded_winger['Scaler']
print(model.feature_names_in_)


input_model = pd.DataFrame(pd.Series(player_attr)).T
input_model = scaler.transform(input_model[model.feature_names_in_])
print(model.predict(input_model))

explainer = loaded_winger['explainer']
pd.DataFrame(index=model.feature_names_in_, data = explainer.shap_values(input_model)[0])

## Central Midfielder

In [ ]:
loaded_winger = pd.read_pickle("prediction_prime_overall/data/models/central_midfielder_prime.pkl")[ 'Gradient Boosting Regressor']
model = loaded_winger['Model']
scaler = loaded_winger['Scaler']
print(model.feature_names_in_)


input_model = pd.DataFrame(pd.Series(player_attr)).T
input_model = scaler.transform(input_model[model.feature_names_in_])
print(model.predict(input_model))

explainer = loaded_winger['explainer']
pd.DataFrame(index=model.feature_names_in_, data = explainer.shap_values(input_model)[0])

# Analyze individual Players

In [ ]:
ID = 259399

loaded_object = pd.read_pickle("prediction_prime_overall/data/models/forward_prime.pkl")[ 'Gradient Boosting Regressor']
model = loaded_object['Model']
scaler = loaded_object['Scaler']
explainer = loaded_object['explainer']
attributes = model.feature_names_in_
player = analyze_individual_ID(ID = ID	,df_raw=df_prospect,attributes=attributes,model = model,scaler=scaler,explainer= explainer)
player.rename(index=CONFIG.plot_feature_mapping, inplace=True)
create_polar_plot(player['shap'], positive_color= "white", negative_color='black')

pos_skills = player.shap[player.shap>0]
np.round(pos_skills/pos_skills.sum(),2)

# Center Forward

In [ ]:
for ID in [268421,259399,240833,251852,256658,256790]:
    player = analyze_individual_ID(ID = ID	,df_raw=df_prospect,attributes=attributes,model = model,scaler=scaler,explainer= explainer)
    player.rename(index=CONFIG.plot_feature_mapping, inplace=True)
    create_polar_plot(player['shap'], positive_color= "white", negative_color='black')
    
    print("-"*150)
    print("#"*150)

# Attacking Midfielder

In [ ]:
import pickle
ACTUAL = "offense_potential_attacking_midfielder_age"
NAME = "attacking_midfielder_prime"
with open(f"{CONFIG.MODELS}/{NAME}.pkl", "rb") as file:
    loaded_object = pickle.load(file)

chosen_object = loaded_object[ 'Gradient Boosting Regressor']
model = chosen_object['Model']
scaler = chosen_object['Scaler']
attributes = chosen_object['attributes']
explainer = chosen_object['explainer']
print(chosen_object['Classification Report'])
# print(chosen_object['AUC'])


In [ ]:
player_attr = {	
    "ShortPassing":88,
 	"Finishing":84,
    "Positioning":85,
    "BallControl":89,
    "Dribbling":87,
    "Reactions":85,
    "Vision":88
}
input_model = pd.DataFrame(pd.Series(player_attr)).T
input_model = scaler.transform(input_model[model.feature_names_in_])
input_model

In [ ]:

model.predict(input_model)

In [ ]:

from config import plot_feature_mapping

for ID in [251854,256630,256790,251852]:
    player = analyze_individual_ID(ID = ID	,df_raw=df_prospect,attributes=attributes,model = model,scaler=scaler,explainer= explainer)
    player.rename(index=plot_feature_mapping, inplace=True)
    create_polar_plot(player['shap'], positive_color= "#5a7b6c", negative_color='#e34234')
    
    print("-"*150)
    print("#"*150)